In [1]:
import xml.etree.ElementTree as ET
import os 

# Parse the XML file
tree = ET.parse('C:\KJB_and_KTR\simplepentaho\simplepentaho\Process all tables.kjb')
root = tree.getroot()

# Find all 'entry' elements
entries = root.findall('.//entry')

# Extract the filenames from each entry
filenames = []
for entry in entries:
    filename_element = entry.find('filename')
    if filename_element is not None:
        filenames.append(filename_element.text)

# Print the extracted filenames
for filename in filenames:
    print(filename)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\KJB_and_KTR\\simplepentaho\\simplepentaho\\Process all tables.kjb'

In [5]:
import os
import xml.etree.ElementTree as ET

# Assume 'filenames' is the list of filenames you obtained from the previous operation
filenames = [
    "${Internal.Job.Filename.Directory}/Get list of tables.ktr",
    "${Internal.Job.Filename.Directory}/Process one table.kjb",
    "${Internal.Job.Filename.Directory}/save list of all result files.ktr"
]

# Replace the placeholder with the actual directory path
job_directory = "C:\KJB_and_KTR\simplepentaho\simplepentaho"  # Replace with the actual directory path

# Process each file
for filename in filenames:
    # Replace the placeholder with the actual directory path
    actual_filename = filename.replace("${Internal.Job.Filename.Directory}", job_directory)
    
    # Check if the file exists
    if os.path.isfile(actual_filename):
        # Parse the XML file
        tree = ET.parse(actual_filename)
        root = tree.getroot()
        
        # Perform your processing on the root element of each file
        # For example, print the root tag name
        print(f"Root tag of '{actual_filename}':", root.tag)
        
        # If you need to find specific elements, use root.findall() or other ElementTree functions
        # ...
    else:
        print(f"File not found: {actual_filename}")


Root tag of 'C:\KJB_and_KTR\simplepentaho\simplepentaho/Get list of tables.ktr': transformation
Root tag of 'C:\KJB_and_KTR\simplepentaho\simplepentaho/Process one table.kjb': job
Root tag of 'C:\KJB_and_KTR\simplepentaho\simplepentaho/save list of all result files.ktr': transformation


In [43]:
import pandas as pd
import json
import openai
import pyodbc
#from openai import AzureOpenAI
import os

In [2]:
import pandas as pd
import json
import openai
import os
import shutil
import xml.etree.ElementTree as ET
import requests
import logging
from logging.handlers import RotatingFileHandler

# Configure logging with a rotating log file handler
log_filename = 'app.log'
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
handler = RotatingFileHandler(log_filename, maxBytes=10*1024*1024, backupCount=1)
handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger = logging.getLogger(__name__)
logger.addHandler(handler)

file_path = None
folder_name = None

def get_elements_from_file(xml_file_path, element_name):
    root = ET.parse(xml_file_path).getroot()
    step_elements = root.findall(f'.//{element_name}')
    return [ET.tostring(step, encoding='unicode') for step in step_elements] if step_elements else []

def save_elements_to_file(input_filename, output_filename):
    xml_file_path = f'{input_filename}'
    output_text_file_path = f'{output_filename}s.txt' if output_filename == 'step' else 'hop_order.txt'
    step_data_strings = get_elements_from_file(xml_file_path, output_filename)

    logger.info(f"{output_filename} elements extracted")

    with open(output_text_file_path, 'w', encoding='utf-8') as output_file:
        output_file.writelines(f"{step_data_string}\n" for step_data_string in step_data_strings)

    logger.info(f"The output has been saved to '{output_text_file_path}'")
    print(f"The output has been saved to '{output_text_file_path}'")

def openai_sequence_steps(hop_order_file_path, steps_file_path):
    # Read the configuration file for OpenAI credentials
    with open('config_file.json') as f:
        json_data = json.load(f)

    # Set up OpenAI configuration with the loaded credentials
    openai.api_type = json_data["openai_api_type"]
    openai.api_base = json_data["openai_api_base"]
    openai.api_version = json_data["openai_api_version"]
    openai.api_key = json_data['openai_api_key']
  
    # Read the contents of the steps file
    with open(hop_order_file_path, 'r', encoding='utf-8') as file:
        hop_order = file.read()
        
    # Read the contents of the steps file
    with open(steps_file_path, 'r', encoding='utf-8') as file:
        steps = file.read()
  
    try:
        # Make the API call to OpenAI with the hop info and the contents of steps.txt
        response = openai.ChatCompletion.create(
            engine="gpt-4-32k",
            temperature=0.1,
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert in converting Pentaho code to Spark code.\n\nYou will receive a hop order file and a steps file. Analyze the hop order to determine the sequence, then convert the corresponding steps into Spark code following that sequence.\n\nReturn the Spark code for the entire flow step by step as per the steps file.\n\nGive sequence number to each step."
                },
                {   "role": "user", 
                    "content": f"Here is the Pentaho hop order file: {hop_order} and the steps file {steps}"
                }
            ],
        )
      
        # Extract the output from the response
        output = response["choices"][0]["message"]["content"]
        return output
    except Exception as e:
        print(e)
        logger.error(f"{e}")
        raise

def copy_specific_files(source_folder, target_folder, file_names):
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    for file_name in file_names:
        source_path = os.path.join(source_folder, file_name)
        target_path = os.path.join(target_folder, file_name)
        if os.path.exists(source_path):
            shutil.copy(source_path, target_path)
            print(f"Copied '{file_name}' to '{target_folder}'.")
        else:
            print(f"File '{file_name}' not found in '{source_folder}'.")




In [3]:

def process_ktr(file_path, folder_name):
        input_filename = file_path
        folder_name = os.path.join('spark_code', folder_name)
        hop_order_file_path = "hop_order.txt"
        steps_file_path = "steps.txt"
        output_file_path = "pyspark_code.txt"

        save_elements_to_file(input_filename, 'order')
        save_elements_to_file(input_filename, 'step')

        code_output = openai_sequence_steps(hop_order_file_path, steps_file_path)
        print(code_output)

        with open(output_file_path, 'a', encoding='utf-8') as output_file:
            output_file.write(code_output)

        source_folder = ""
        target_folder = folder_name
        file_names = ["hop_order.txt", "steps.txt", "pyspark_code.txt"]
        copy_specific_files(source_folder, target_folder, file_names)
        for file_name in file_names:
            file_path = os.path.join(source_folder, file_name)
            os.remove(file_path)
        logger.info(f"The converted code is saved in {folder_name} folder")


In [4]:
def process_directory(directory_path):
    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            # Process the file based on its extension
            if file_path.endswith('.ktr'):
                print('file_path is', file_path)
                file_name = os.path.basename(file_path)
                folder_name = os.path.splitext(file_name)[0]
                process_ktr(file_path, folder_name)

# Start the processing with the main directory
main_directory_path = r'C:\KJB_and_KTR\simplepentaho'  # Replace with the path to your directory
process_directory(main_directory_path)

2024-06-11 11:31:14,043 - INFO - order elements extracted
2024-06-11 11:31:14,046 - INFO - The output has been saved to 'hop_order.txt'
2024-06-11 11:31:14,048 - INFO - step elements extracted
2024-06-11 11:31:14,051 - INFO - The output has been saved to 'steps.txt'


file_path is C:\KJB_and_KTR\simplepentaho\Get list of tables.ktr
The output has been saved to 'hop_order.txt'
The output has been saved to 'steps.txt'


2024-06-11 11:32:03,136 - INFO - The converted code is saved in spark_code\Get list of tables folder
2024-06-11 11:32:03,149 - INFO - order elements extracted
2024-06-11 11:32:03,152 - INFO - The output has been saved to 'hop_order.txt'
2024-06-11 11:32:03,156 - INFO - step elements extracted
2024-06-11 11:32:03,159 - INFO - The output has been saved to 'steps.txt'


Based on the hop order and the steps file, the sequence of steps is as follows:

1. get list of tables
2. Concat Fields
3. Select values
4. Copy rows to result
5. tables.txt

Now, let's convert these steps into Spark code:

1. get list of tables

```scala
// Step 1: get list of tables
val jdbcDF = spark.read
  .format("jdbc")
  .option("url", "jdbc:mysql://localhost:3306/database")
  .option("dbtable", "(SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME NOT LIKE 'SYSTEM_%') as tables")
  .option("user", "username")
  .option("password", "password")
  .load()
```

2. Concat Fields

```scala
// Step 2: Concat Fields
import org.apache.spark.sql.functions.concat_ws
val concatDF = jdbcDF.withColumn("TABLE_SCHEMA_NAME", concat_ws(".", $"TABLE_SCHEMA", $"TABLE_NAME"))
```

3. Select values

```scala
// Step 3: Select values
val selectDF = concatDF.select($"TABLE_SCHEMA_NAME".as("tablename"))
```

4. Copy rows to result

```scala
// Step 4: Copy rows to result
// In Spark, this step is 

2024-06-11 11:32:38,864 - INFO - The converted code is saved in spark_code\Process one table folder
2024-06-11 11:32:38,876 - INFO - order elements extracted
2024-06-11 11:32:38,879 - INFO - The output has been saved to 'hop_order.txt'
2024-06-11 11:32:38,882 - INFO - step elements extracted
2024-06-11 11:32:38,884 - INFO - The output has been saved to 'steps.txt'


Based on the hop order and steps file, the sequence of steps is as follows:

1. Number of rows in ${TABLENAME}
2. rows-${TABLENAME}.txt

Now, let's convert these steps into Spark code:

Step 1: Number of rows in ${TABLENAME}

This step is a TableInput type, which means it's reading data from a table. The SQL query is "SELECT count(*) as NrRows FROM ${TABLENAME}". This can be converted into Spark code as follows:

```scala
val tableName = "your_table_name" // replace with your table name
val df = spark.sql(s"SELECT count(*) as NrRows FROM $tableName")
```

Step 2: rows-${TABLENAME}.txt

This step is a TextFileOutput type, which means it's writing the output to a text file. The file name is "${java.io.tmpdir}/rows-${TABLENAME}.txt". This can be converted into Spark code as follows:

```scala
val outputDir = System.getProperty("java.io.tmpdir")
df.coalesce(1).write.format("text").option("header", "true").save(s"$outputDir/rows-$tableName.txt")
```

So, the complete Spark code is:

```scal

2024-06-11 11:32:59,171 - INFO - The converted code is saved in spark_code\save list of all result files folder
2024-06-11 11:32:59,174 - INFO - order elements extracted
2024-06-11 11:32:59,176 - INFO - The output has been saved to 'hop_order.txt'
2024-06-11 11:32:59,178 - INFO - step elements extracted
2024-06-11 11:32:59,180 - INFO - The output has been saved to 'steps.txt'


Based on the hop order and steps file, the sequence of steps is as follows:

1. Get files from result
2. Text file output

Now, let's convert these steps into Spark code:

Step 1: Get files from result
In Pentaho, this step is used to get the list of files from the previous result. In Spark, we can use the wholeTextFiles method to get the files from a directory.

```scala
val files = sc.wholeTextFiles("path_to_directory")
```

Step 2: Text file output
In Pentaho, this step is used to write the result into a text file. In Spark, we can use the saveAsTextFile method to save the result into a text file.

```scala
files.saveAsTextFile("${java.io.tmpdir}files")
```

Please replace "path_to_directory" with the actual directory path where the files are located. Also, "${java.io.tmpdir}files" is the output directory where the result will be saved. If you want to save the result in a different directory, please replace "${java.io.tmpdir}files" with your preferred directory path.
Copied 'hop_ord

2024-06-11 11:33:32,220 - INFO - The converted code is saved in spark_code\set variables folder


Based on the hop order file and the steps file, the sequence of steps is as follows:

1. Get one tablename
2. Set ${TABLENAME}

Now, let's convert these steps into Spark code.

1. Get one tablename

In Pentaho, the "RowsFromResult" step is used to get the result of a previous step. In Spark, we can use the "collect()" function to get the result of a previous transformation. Assuming that the previous transformation is a DataFrame named "df", the equivalent Spark code would be:

```scala
val tablename = df.collect()
```

2. Set ${TABLENAME}

In Pentaho, the "SetVariable" step is used to set a variable that can be used in subsequent steps. In Spark, we can simply assign the result of the previous step to a variable. Assuming that the result of the previous step is stored in the variable "tablename", the equivalent Spark code would be:

```scala
val TABLENAME = tablename
```

So, the complete Spark code for the entire flow would be:

```scala
val tablename = df.collect()
val TABLENAME = t

In [9]:
file_path = r'C:\KJB_and_KTR\simplepentaho\Get list of tables.ktr'
file_name = os.path.basename(file_path)
folder_name = os.path.splitext(file_name)[0]
folder_name

'Get list of tables'

In [40]:
folder_name = os.path.splitext(file_name)[0]
folder_name = os.path.splitext(file_names)[0]
print(folder_name)

In [5]:
import os# Base directory and filename
base_dir = 'user'
filename = 'example'
 
# Construct the full path
full_path = os.path.join('spark_code', filename)
full_path


'spark_code\\example'

In [39]:
import os

# Base directory where the .ktr files are located
base_dir = "C:/KJB_and_KTR/simplepentaho"

# List of .ktr files
ktr_files = []

# Function to create a folder for each .ktr file
def create_folders_for_ktr(files, base_directory):
    for file in files:
        # Extract the folder name by removing the '.ktr' extension
        folder_name = os.path.splitext(file)[0]
        # Create the full path for the new folder
        folder_path = os.path.join(base_directory, folder_name)
        # Create the folder if it doesn't exist
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Folder created: {folder_path}")
        else:
            print(f"Folder already exists: {folder_path}")

# Call the function to create folders
create_folders_for_ktr(ktr_files, base_dir)

# Add your code here to save content in each folder
# For example, to create a text file in each folder:
for file in ktr_files:
    folder_name = os.path.splitext(file)[0]
    folder_path = os.path.join(base_dir, folder_name)
    content_file_path = os.path.join(folder_path, "example.txt")
    with open(content_file_path, "w") as content_file:
        content_file.write("This is an example of content saved in the folder.")
        print(f"Content saved in: {content_file_path}")


In [13]:
import os
import xml.etree.ElementTree as ET

def process_ktr(file_path):
   
    def openai_sequence_hops_pentaho_logic(question):
        f = open('config_file.json')
        json_data = json.load(f)

        openai.api_type = json_data["openai_api_type"]
        openai.api_base = json_data["openai_api_base"]
        openai.api_version = json_data["openai_api_version"]
        openai.api_key = json_data['openai_api_key']
    
        try:
            response = openai.ChatCompletion.create(
                engine="gpt-4-32k",
                temperature=0.1,
                messages= [
                    {
                        "role": "system",
                        "content": '''You are an expert pentaho developer. Analyze the provided KJB file. And give me the logic for the KJB File '''
    
                    },
                    {"role": "user", "content": f"{question}"},
                ]
            )
        
            output = response["choices"][0]["message"]["content"]
            return output
            
        except Exception as e:
            print(e)
            raise
    print(f"Processing ARM logic for: {file_path}")

def process_kjb(file_path):
    
    def openai_sequence_steps(hop_order_file_path, steps_file_path):
    # Read the configuration file for OpenAI credentials
        with open('config_file.json') as f:
            json_data = json.load(f)

        # Set up OpenAI configuration with the loaded credentials
        openai.api_type = json_data["openai_api_type"]
        openai.api_base = json_data["openai_api_base"]
        openai.api_version = json_data["openai_api_version"]
        openai.api_key = json_data['openai_api_key']
    
        # Read the contents of the steps file
        with open(hop_order_file_path, 'r', encoding='utf-8') as file:
            hop_order = file.read()
            
        # Read the contents of the steps file
        with open(steps_file_path, 'r', encoding='utf-8') as file:
            steps = file.read()
    
        try:
            # Make the API call to OpenAI with the hop info and the contents of steps.txt
            response = openai.ChatCompletion.create(
                engine="gpt-4-32k",
                temperature=0.1,
                messages=[
                    {
                        "role": "system",
                        "content": "You are an expert in converting Pentaho code to Spark code.\n\nYou will receive a hop order file and a steps file. Analyze the hop order to determine the sequence, then convert the corresponding steps into Spark code following that sequence.\n\nReturn the Spark code for the entire flow step by step as per the steps file.\n\nGive sequence number to each step."
                    },
                    {   "role": "user", 
                        "content": f"Here is the Pentaho hop order file: {hop_order} and the steps file {steps}"
                    }
                ],
            )
        
            # Extract the output from the response
            output = response["choices"][0]["message"]["content"]
            return output
        except Exception as e:
            print(e)

    print(f"Processing PySpark code for: {file_path}")

def process_directory(directory_path):
    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)

        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            # Process the file based on its extension
            if file_path.endswith('.ktr'):
                process_ktr(file_path)
            elif file_path.endswith('.kjb'):
                process_kjb(file_path)

# Start the processing with the main directory
main_directory_path = 'C:\KJB_and_KTR\simplepentaho'  # Replace with the path to your directory
process_directory(main_directory_path)


In [8]:
print(process_directory)

<function process_directory at 0x00000198F2C02B60>


In [12]:
def process_directory(directory_path):
    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)

        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            # Process the file based on its extension
            if file_path.endswith('.ktr'):
                process_ktr(file_path)
            elif file_path.endswith('.kjb'):
                process_kjb(file_path)

In [ ]:

def process_ktr(file_path):
        input_filename = file_path
        folder_name = 'spark_code/' + folder_name
        hop_order_file_path = "hop_order.txt"
        steps_file_path = "steps.txt"
        output_file_path = "pyspark_code.txt"

        save_elements_to_file(input_filename, 'order')
        save_elements_to_file(input_filename, 'step')

        code_output = openai_sequence_steps(hop_order_file_path, steps_file_path)
        print(code_output)

        with open(output_file_path, 'a', encoding='utf-8') as output_file:
            output_file.write(code_output)

        source_folder = ""
        target_folder = folder_name
        file_names = ["hop_order.txt", "steps.txt", "pyspark_code.txt"]
        copy_specific_files(source_folder, target_folder, file_names)
        for file_name in file_names:
            file_path = os.path.join(source_folder, file_name)
            os.remove(file_path)
        logger.info(f"The converted code is saved in {folder_name} folder")

        folder_path = 'uploads'
        files = os.listdir(folder_path)
        for file_name in files:
            file_path = os.path.join(folder_path, file_name)
            os.remove(file_path)
